In [1]:
import numpy as np
import pandas
#df2 = pandas.read_csv('test.csv', encoding='latin-1')

In [2]:
df = pandas.read_csv('train.csv', encoding='latin-1', names=["Sentiment", "ID", "Date", "Query","Username","Text"])
df = df.drop(["ID", "Date", "Query","Username"], axis=1)

In [3]:
df['Text'] = df['Text'].str.replace("&gt;_&lt;", "sad")
df['Text'] = df['Text'].str.replace(";-;", "sad")
df['Text'] = df['Text'].str.replace(";_;", "sad")
df['Text'] = df['Text'].str.replace(" D: ", " sad ")
df['Text'] = df['Text'].str.replace(" :D ", " happy ")
df['Text'] = df['Text'].str.replace(" ;D", " happy")
df['Text'] = df['Text'].str.replace("xd", "laugh")
df['Text'] = df['Text'].str.replace("=.=", "confused")
df['Text'] = df['Text'].str.replace("o_o", "confused")
df['Text'] = df['Text'].str.replace("O_o", "confused")

df['Text'] = df['Text'].str.lower()

#df['Text'] = df['Text'].str.replace("...", " ")
df["Text"] = df["Text"].str.replace('(\@\w+.*?)',"")
df["Text"] = df["Text"].str.replace('(\#\w+.*?)',"")
df['Text'] = df['Text'].str.replace('http[^\s]*',"")
df['Text'] = df['Text'].str.replace(".", "")
df['Text'] = df['Text'].str.replace(",", "")
df['Text'] = df['Text'].str.replace("-", "")
df['Text'] = df['Text'].str.replace("`", "")
df['Text'] = df['Text'].str.replace("'", "")
df['Text'] = df['Text'].str.replace(": ", " ")
df['Text'] = df['Text'].str.replace("; ", " ")
df['Text'] = df['Text'].str.replace("&amp;", "and")
df['Text'] = df['Text'].str.replace("&quot;", "and")
df['Text'] = df['Text'].str.replace("?", "")
df['Text'] = df['Text'].str.replace("!", "")
df['Text'] = df['Text'].str.replace("*", "")
df["Text"] = df["Text"].str.replace('ï¿½',"")
df['Text'] = df['Text'].str.replace("&amp", "")
df['Text'] = df['Text'].str.replace("&quot", "")
df['Text'] = df['Text'].str.replace("&gt", "")
df['Text'] = df['Text'].str.replace("&lt", "")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace("  ", " ")
df['Text'] = df['Text'].str.replace("  ", " ")

df['Text'] = df['Text'].str.replace(" @ ", " ")
df['Text'] = df['Text'].str.replace("upits", "up its")
df['Text'] = df['Text'].str.replace("Of to ", "off to ")
df['Text'] = df['Text'].str.replace("brightside", "bright side")
df['Text'] = df['Text'].str.replace("thaanks", "thanks")
df['Text'] = df['Text'].str.replace("thaaanks", "thanks")

df['Text'] = df['Text'].str.lstrip(' ')
df['Text'] = df['Text'].str.rstrip(' ')



C:\Users\hyun9\AppData\Local\Temp\ipykernel_16556\883629432.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace("=.=", "confused")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_16556\883629432.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Text"] = df["Text"].str.replace('(\@\w+.*?)',"")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_16556\883629432.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Text"] = df["Text"].str.replace('(\#\w+.*?)',"")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_16556\883629432.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace('http[^\s]*',"")
C:\Users\hyun9\AppData\Local\Temp\ipykernel_16556\883629432.py:18: FutureWarning: The default value of regex will change from True to F

In [4]:
print(df["Text"][2])
print(df["Text"][20])

i dived many times for the ball managed to save 50% the rest go out of bounds
i was out most of the day so didnt get much done


In [5]:
# Dataset

import torch
import numpy as np
from transformers import BertTokenizer

MAXLENGTH = 512


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels for labels in df['Sentiment']]
        self.texts = [tokenizer(text, padding='max_length', max_length = MAXLENGTH, truncation=True, return_tensors="pt") for text in df['Text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y

In [6]:
np.random.seed(42)
trainX, evaluationX, testX = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])

In [7]:
# Reduce Dataset Size (Temp)

trainX = trainX.drop(trainX.index[0:1035000])
testX = testX.drop(testX.index[0:159501])
evaluationX = evaluationX.drop(evaluationX.index[0:159501])

In [8]:
trainX.shape, testX.shape, evaluationX.shape

((245000, 2), (499, 2), (499, 2))

In [9]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5) # CHANGE TO 2? / entropy classifier might also have to change
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [29]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5 # ---------------
model = BertClassifier()
LR = 1e-6

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train(model, trainX, evaluationX, LR, EPOCHS)

In [28]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print(input_id.shape)
    print(mask.shape)
    print(mask)

In [31]:
model_1 = BertClassifier()
model_1.load_state_dict(torch.load("C:/Users/hyun9/Desktop/New folder (2)/model.pth"))
model_1 = model_1.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
softmax = torch.nn.Softmax(dim=None)

In [58]:
text = "strep throat first day = super no bueno"
a = tokenizer(text, padding='max_length', max_length = MAXLENGTH, truncation=True, return_tensors="pt")
softmax(model_1(a["input_ids"].to(device), a["attention_mask"].reshape([1,1,512]).to(device)))

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 5.33 GiB already allocated; 0 bytes free; 5.34 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [59]:
num = 57
print(trainX.iloc[num, 0])
trainX.iloc[num, 1]

4


'i heard the results are worth it but im such a baby lol i plan to try it at least once though'